In [1]:
#This will essentially be just like neuralOrbitTF but I will do some custom training
# Hopefully this will give me the accuracy I am looking for 

from orbitPropagator import (
    twoBodyProp,
    cartesianToOrbitalElements,
    orbitalElementsToCartesian,
)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from orbitPlotter import plotter
import keras_tuner as kt

In [ ]:
## Here we will initialize our datasets

# Important Constants 
G = 6.67 * 10**-11  # N*m^2/kg^2
m_earth = 5.972 * 10**24  # kg
altitude_test = 8 * 10**6  # m
altitude_train = 5 * 10**6  # m
planet_radius = 6.371 * 10**6  # Earth radius meters

# First I'll create a suitable list of orbits using Keplerian orbital elements
orbits = 200
orbital_set_initial_conditions = np.zeros((orbits, 6))

## Using random I will populate each of these with acceptable parameters
## Rules - semimajor axis (a) must be in the range of 2,000 km to ~36,000 km
##      - inclination must be between 0 and pi
##      - eccentricity must be between 0 and 1
##      - RAAN is 0 and 2pi
##      - periapsis is 0 and 2pi
##      - true anomaly is 0 and 2pi
# semi = np.linspace(2000000 + planet_radius, 36000000 + planet_radius, orbits) ## for testing if I need structured training
for i in range(orbits):
    for j in range(6):
        if j == 0:
            #Everything I've done is in meters
            semimajor = np.random.randint(low=2000000 + planet_radius, high=36000000 + planet_radius)
            orbital_set_initial_conditions[i, j] = semimajor
        
        if j == 1:
            ## I'll do eccentricities between 0 and 0.1
            e = np.random.rand() * 0.1  # random may not be so good lol (unless your sample size is large enough, lets make it larger)
            orbital_set_initial_conditions[i, j] = e
        
        if j == 2 :
            inclination = np.random.rand() * np.pi
            orbital_set_initial_conditions[i, j] = inclination
        if j > 2:
            random = 2 * np.pi * np.random.rand()
            orbital_set_initial_conditions[i, j] = random

In [ ]:
## Now that we have our numerically well conditioned orbits between LEO and GEO, we can propagate them
## for now all but the last will be for training

## We will have to convert each back to cartesian, propagate, and then back to orbital elements with the associated time step
propagated_orbits = []
cartesian_propagated_array = []
for i in range(orbits):
    cartesian_orbit = orbitalElementsToCartesian(orbital_set_initial_conditions[i], m_earth)
    cartesian_propagated = twoBodyProp(
        cartesian_orbit[0], 
        cartesian_orbit[1],
        cartesian_orbit[2],
        cartesian_orbit[3],
        cartesian_orbit[4],
        cartesian_orbit[5],
        export_time=True,
        time_step=50
        )
    for j in range(len(cartesian_propagated)):
        # Back to orbital elements
        cartesian_propagated[j] = cartesianToOrbitalElements(cartesian_propagated[j], m_earth, isTime=True)
    
    propagated_orbits.append(cartesian_propagated)
# Now what we have is a list of numpy arrays containing the orbital elements for each of the defined orbits including time steps 

## Now we need to split these orbits into training and test sets
## Create the training sets without true anomaly
train_features = [np.delete(arr, 5, axis=1) for arr in propagated_orbits[:int(orbits-(orbits*.2))]]
train_labels = [arr[:, 5] for arr in propagated_orbits[:int(orbits-(orbits*.2))]]

test_features = [np.delete(arr, 5, axis=1) for arr in propagated_orbits[int(orbits-(orbits*.2)):]]
test_labels = [arr[:, 5] for arr in propagated_orbits[int(orbits-(orbits*.2)):]]

test_features_single = np.delete(propagated_orbits[orbits-1], 5, axis=1)
test_labels_single = propagated_orbits[orbits-1][:, 5]

## Now we normalize it
## this is where the mishapen nature of train_features has caught up with me
## Up until now it was good because it kept orbits separated but now I really should just squash it together
train_features = np.vstack(train_features)
flat_array = np.concatenate(train_labels)
# Reshape to (number of values, 1)
train_labels = flat_array.reshape(-1, 1)
test_features = np.vstack(test_features)
flat_array2 = np.concatenate(test_labels)
test_labels = np.vstack(flat_array2)

test_features_single = np.vstack(test_features_single)
test_labels_single = np.vstack(test_labels_single)

## Now everything is happy
train_features_normalizer = layers.Normalization(axis=-1)
train_features_normalizer.adapt(train_features)